**Data Proprecess**

import several libraries that I am going to use, including.

In [ ]:
import os
import nltk
import numpy as np
import pandas as pd
import sklearn
import operator
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from google.colab import drive
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

Obtain all Articles and Content

In [ ]:
# Get all data and categories them
ori_data=[]
for c in category:
  for file_name in os.listdir(dataset_path + "/" + c):
    file = open(dataset_path + "/" + c + "/" +file_name, "r")
    ori_data.append([file.read(), c])
    file.close()
All_data = pd.DataFrame(ori_data, columns=["content","category"])
All_data.shape
x_data = All_data.iloc[: , 0]
y_data = All_data.iloc[: , -1]

Split all the original data

In [ ]:
# Split all the original data into three different set, including train set (80%), development set (10%) and test set (10%).
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=1)
x_dev, x_test, y_dev, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=1)

Remove words that are not useful for classification and unified format

In [ ]:
stopwords=set(nltk.corpus.stopwords.words('english'))
stopwords.add(".")
stopwords.add(",")
stopwords.add("-")
stopwords.add("``")
stopwords.add("/")
stopwords.add("(")
stopwords.add(")")
stopwords.add("{")
stopwords.add("}")
stopwords.add("@")
stopwords.add("|")
stopwords.add(";")
stopwords.add("\n")
stopwords.add("#")
stopwords.add("+")
stopwords.add("_")
stopwords.add("''")
stopwords.add(":")
stopwords.add("%")
stopwords.add("$")
stopwords.add("&")
stopwords.add("'")

lemmatizer = nltk.stem.WordNetLemmatizer()

**Feature Selection**

Feature 1: Article Word Frequency

In [ ]:
# Get sorted vocabulary list of feature 1
word_fre = {}
for article in x_train:
  sentences = nltk.tokenize.sent_tokenize(article)
  for sent in sentences:
    tokens = nltk.tokenize.word_tokenize(sent)
    for token in tokens:
      word = lemmatizer.lemmatize(token).lower()
      if word in stopwords: continue
      if word in word_fre: word_fre[word] += 1
      else: word_fre[word] = 1
sorted_list = sorted(word_fre.items(), key = operator.itemgetter(1), reverse = True)

Feature 2: Article Title Word Frequency

In [ ]:
# Get sorted vocabulary list of feature 2
title_words_fre = {}
for article in x_train:
  sentences = nltk.tokenize.sent_tokenize(article)
  tokens = nltk.tokenize.word_tokenize(sentences[0])
  for token in tokens:
      word = lemmatizer.lemmatize(token).lower()
      if word in stopwords: continue
      if word in title_words_fre: title_words_fre[word] += 1
      else: title_words_fre[word] = 1
sorted_list = sorted(title_words_fre.items(), key = operator.itemgetter(1), reverse = True)

Feature 3: TF-IDF (term frequency-inverse document frequency)

In [ ]:
# initialize all data to tf-idf Vectorizer and transform documents of train set and test set to matrix.
tfidf_vector = TfidfVectorizer()
tfidf_vector.fit(x_train)
tfidf_train_x = tfidf_vector.transform(x_train)
tfidf_test_x  = tfidf_vector.transform(x_test)

**Train, Test and Performance**

Feature 1

In [ ]:
cur_size = 20
vocabulary = []
accuracy = 0
svm_clf_category = sklearn.svm.SVC(kernel="linear",gamma="auto")
while accuracy < 0.9:
    # generate vocabulary
    vocabulary = []
    for word,fre in sorted_list[:cur_size]:
        vocabulary.append(word)
    
    # prepare x and y for model fit
    x = []
    y = []
    for index in x_train.index:
        article = x_train[index]
        vector = np.zeros(len(vocabulary))
        words=[]
        sentences = nltk.tokenize.sent_tokenize(article)
        for sent in sentences:
            tokens = nltk.tokenize.word_tokenize(sent)
            for token in tokens:
              word = lemmatizer.lemmatize(token).lower()
              words.append(word)
        for i, word in enumerate(vocabulary):
            if word in words:
                vector[i] =words.count(word)
        x.append(vector)
        y.append(y_train[index])
    
    # fit model
    svm_clf_category.fit(np.asarray(x), np.asarray(y))
    
    # prepare dev set for predictions
    x = []
    y = []
    for index in x_dev.index:
        article = x_dev[index]
        vector = np.zeros(len(vocabulary))
        words=[]
        sentences = nltk.tokenize.sent_tokenize(article)
        for sent in sentences:
          tokens = nltk.tokenize.word_tokenize(sent)
          for token in tokens:
            word = lemmatizer.lemmatize(token).lower()
            words.append(word)
        for i, word in enumerate(vocabulary):
            if word in words:
                vector[i] =words.count(word)
        x.append(vector)
        y.append(y_dev[index])
    
    # predict accuracy
    prediction = svm_clf_category.predict(np.asarray(x))
    y = np.asarray(y)
    accuracy = accuracy_score(y, prediction)
    print("cur_size: " + str(cur_size) + ", accuracy: " + str(accuracy))
    
    # increase current size for better accuracy
    cur_size += 20

Performance

In [ ]:
x = []
y = []
for index in x_test.index:
  article = x_test[index]
  vector = np.zeros(len(vocabulary))
  words=[]
  sentences = nltk.tokenize.sent_tokenize(article)
  for sent in sentences:
    tokens = nltk.tokenize.word_tokenize(sent)
    for token in tokens:
      word = lemmatizer.lemmatize(token).lower()
      words.append(word)
  for i, word in enumerate(vocabulary):
    if word in words:
      vector[i] =words.count(word)
  x.append(vector)
  y.append(y_test[index])
prediction = svm_clf_category.predict(np.asarray(x))
y = np.asarray(y)

Precision=92.0%, Recall=92.4%, F1=92.1%, Accuracy=92.4%

Feature 2

In [ ]:
# use dev set to find a appropriate size of vocabulary
cur_size = 20
vocabulary = []
accuracy = 0
svm_clf_category = sklearn.svm.SVC(kernel="linear",gamma="auto")
while accuracy < 0.9:
    # generate vocabulary
    vocabulary = []
    for word,fre in sorted_list[:cur_size]:
        vocabulary.append(word)
    
    # prepare x and y for model fit
    x = []
    y = []
    for index in x_train.index:
        article = x_train[index]
        vector = np.zeros(len(vocabulary))
        words=[]
        sentences = nltk.tokenize.sent_tokenize(article)
        for sent in sentences:
            tokens = nltk.tokenize.word_tokenize(sent)
            for token in tokens:
              word = lemmatizer.lemmatize(token).lower()
              words.append(word)
        for i, word in enumerate(vocabulary):
            if word in words:
                vector[i] =words.count(word)
        x.append(vector)
        y.append(y_train[index])
    
    # fit model
    svm_clf_category.fit(np.asarray(x), np.asarray(y))
    
    # prepare dev set for predictions
    x = []
    y = []
    for index in x_dev.index:
        article = x_dev[index]
        vector = np.zeros(len(vocabulary))
        words=[]
        sentences = nltk.tokenize.sent_tokenize(article)
        for sent in sentences:
          tokens = nltk.tokenize.word_tokenize(sent)
          for token in tokens:
            word = lemmatizer.lemmatize(token).lower()
            words.append(word)
        for i, word in enumerate(vocabulary):
            if word in words:
                vector[i] =words.count(word)
        x.append(vector)
        y.append(y_dev[index])
    
    # predict accuracy
    prediction = svm_clf_category.predict(np.asarray(x))
    y = np.asarray(y)
    accuracy = accuracy_score(y, prediction)
    print("cur_size: " + str(cur_size) + ", accuracy: " + str(accuracy))
    
    # increase current size for better accuracy
    cur_size += 20

Performance

In [ ]:
# use test set for final score
x = []
y = []
for index in x_test.index:
  article = x_test[index]
  vector = np.zeros(len(vocabulary))
  words=[]
  sentences = nltk.tokenize.sent_tokenize(article)
  for sent in sentences:
    tokens = nltk.tokenize.word_tokenize(sent)
    for token in tokens:
      word = lemmatizer.lemmatize(token).lower()
      words.append(word)
  for i, word in enumerate(vocabulary):
    if word in words:
      vector[i] =words.count(word)
  x.append(vector)
  y.append(y_test[index])
prediction = svm_clf_category.predict(np.asarray(x))
y = np.asarray(y)

Precision=88.7%, Recall=88.8%, F1=88.7%, Accuracy=89.2%

Feature 3

In [ ]:
tfidf_vector = TfidfVectorizer()
tfidf_vector.fit(x_train)
tfidf_train_x = tfidf_vector.transform(x_train)
tfidf_test_x  = tfidf_vector.transform(x_test)
svm_clf_category = sklearn.svm.SVC(kernel="linear",gamma="auto")
svm_clf_category.fit(tfidf_train_x, y_train.to_numpy())

Performance

In [ ]:
svm_clf_category.fit(tfidf_train_x, y_train.to_numpy())
prediction = svm_clf_category.predict(tfidf_test_x)
tfidf_test_y = y_test.to_numpy()

Precision=97.6%, Recall=97.8%, F1=97.7%, Accuracy=97.8%